In [1]:
! ipynb-py-convert C:\jupyter\KDT_AIDEV\finalproject\crawl_baseline_ver01.py C:\jupyter\KDT_AIDEV\finalproject\crawl_baseline_ver01\a.ipynb

'ipynb-py-convert'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [ ]:
acc = 150 # 인터넷,it,통신,모바일,게임 

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

import re

In [9]:
incruit_occu_list = ['웹프로그래머', '응용프로그래머', '시스템프로그래머', '데이터베이스', '데이터베이스·DBA', '네트워크·서버·보안', 'ERP·시스템분석·설계',
            '컨텐츠·사이트운영', '웹디자인', '통신·모바일', '소프트웨어·하드웨어', '전산·IT기술지원', '멀티미디어(동영상·음향)', '게임(Game)',
            '빅데이터·AI(인공지능)', 'HTML·퍼블리싱·UI개발']

incruit_occu_code = [574, 375, 579, 405, 390, 435, 420, 598, 410, 530, 578, 400, 415, 535, 632, 633]

In [17]:
#창 띄우기

chrome_options = Options()
chrome_options.add_experimental_option("detach", True) #창을 보면서 작업할 수 있음
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.maximize_window()

In [7]:
url = "https://job.incruit.com/jobdb_list/searchjob.asp?cate=occu"

def get_job_list(url):

    job_list = []

    driver.get(url)
    button_open = driver.find_element(By.XPATH, '//*[@id="occ1_list_150"]') #인터넷It통신모바일게임 직종 선택
    button_open.click()
    driver.implicitly_wait(2)

    # 직종 클릭 및 초기화버튼은 없어도 스크래핑하는데 문제는 없음 따라서 나중에 필요하다면 없애고 코드를 간결화 할 수 있음
    # 현 단계에서는 잘 돌아가는지 확인하기 위해 클릭 단계를 진행
    i = 0
    for code in incruit_occu_code:
        i += 1

        button_open = driver.find_element(By.XPATH, f'//*[@id="occ2_list_150_{code}"]') #세부직종 선택
        button_open.click()
        driver.implicitly_wait(3)

        # 해당 세부직종 코드 안의 직무 개수 반환
        tmp = driver.find_elements(By.ID, f'occ3_ul_{code}') 
        job_num = len(tmp[0].text.split('\n'))

        for n in range(2, job_num+1):
            job = driver.find_elements(By.XPATH, f'/html/body/div[2]/div[1]/div/div[1]/div/div/div/div[2]/div[1]/div[2]/div[4]/ul/li[{n}]/label')
            driver.implicitly_wait(3)

            job_name = re.sub(r" \(\d+\)", '', job[0].text) # 직무명, (개수)는 없앰
            job_code = int(job[0].get_attribute('data-item').replace('occ3_', '')) #직무코드

            job_list.append((job_name, job_code))


        if i == 8: #인크루트는 직종 선택을 10개를 초과하지 못하게 되어있기 때문에 그 전에 초기화를 눌러서 해소
            button_open = driver.find_element(By.XPATH, '//*[@id="incruit_contents"]/div[1]/div/div/div/div[3]/div[1]/button[1]')
            button_open.click()
            driver.implicitly_wait(2)

            i = 0
    
    return dict(job_list)



In [3]:
cond = {'고용형태' : 1, '경력' : 2, '근무지역' : 3, '학력' : 4, '급여조건' : 5, '우대사항' : 6}

def check_condition(condition): #
    
    i = cond[condition]
    
     # i값에 대해 실행이 되는지 확인
    try:
        driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[{i}]')
        
    except:
        # i값이 없어 에러가 났다면 None을 return
        return None
    
    # i값에 대해 실행이 잘 되었다면 다음을 수행
    else: 
        #버튼을 클릭해 나오는 페이지가 있는지를 확인해서 있다면 버튼 클릭시 나오는 추가 텍스트를 추출
        try:
            botton = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[{i}]/div[2]/div/button')
            botton.click()

            #list형태로 반환이 되므로 join을 해주어야 dataFrame을 만들 때 두세줄씩 추가가 안됨
            result = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[{i}]/div[2]/div/div/ul').text.split('\n')
            result = ', '.join(result)
        #버튼페이지가 없다면 텍스트 추출
        except:
            result = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[{i}]/div[2]').text
        
        #추출 후 다른곳을 클릭해서 열었던 페이지를 지운다.
        finally:
            button = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[4]/div[1]/em')
            button.click()
        
        return result


In [4]:
def image_recruit():
    urllist = []
    
    try:
        els = driver.find_elements(By.TAG_NAME, 'img')
    
    except:
        return None
    
    else:
        for el in els:
            urllist.append(el.get_attribute('src'))
    
        return ', '.join(urllist) if urllist else None

In [5]:
def text_recruit():
    
    txt = driver.find_element(By.XPATH, '/html/body/div').text
    txt = txt.replace('\n', ' ')
    return txt

In [6]:
def url_recruit():
    pass
    return

In [10]:
jl = get_job_list(url)


In [16]:
df = pd.DataFrame(columns=['직무명', '채용공고명', '해시태그','채용공고상세_text', '채용공고상세_img', '채용공고상세_url', '고용형태', '경력', '근무지역', '학력', '급여조건', '우대사항',
                            '회사명', '회사분야', '출처'])

In [18]:

#긁어올 메인페이지 띄우고 main_url경로 초기화
driver.get("https://job.incruit.com/jobdb_list/searchjob.asp?occ3=17111&articlecount=10")
driver.implicitly_wait(3)
current_main_url = driver.current_url

p = 1 
while True:
    print('-'*10, f'{p}page', '-'*10)
    if p != 1: # page가 1이 아니면
        try:
            driver.get(current_main_url)
            driver.implicitly_wait(3)
            
            driver.find_element(By.CLASS_NAME, 'next_n').click()
            driver.implicitly_wait(3)
            
        #끝 페이지에 도달하게 되면 while문 종료    
        except:
            break
        
    # 현재 메인 페이지(공고리스트) url저장
    current_main_url = driver.current_url
    
    # 해당 페이지의 공고 개수
    
    if p == 1:
        j = 2
        
    else:
        j = 1

    num = len(driver.find_elements(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul'))
    
    for i in tqdm(range(1, num + 1)):

        driver.get(current_main_url)
        driver.implicitly_wait(3)
        
        recruit_url = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[2]/div[1]/a').get_attribute('href')
        head = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[2]').text.split('\n')[0][:-4] # 뒤에 스크랩 붙는거 빼주는 용도의 slice
        job_tag = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[2]').text.split('\n')[2]

        c = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[1]').text.split('\n')
        corp = re.sub(r" (관심기업)", '', c[0])
        
        # 나중에 공백 split 해줌
        if len(c) >= 2:
            corp_tag = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[1]').text.split('\n')[1]
        else:
            corp_tag = ''

        driver.implicitly_wait(1)

        driver.get(recruit_url)
        driver.implicitly_wait(2)
        
        #공고 없음의 에러이미지가 있다면 넘어간다.
        if driver.find_elements(By.CLASS_NAME, 'error-img'):
            continue

        emptype = check_condition('고용형태')
        time.sleep(0.5)

        career = check_condition('경력')
        time.sleep(0.5)

        location = check_condition('근무지역')
        time.sleep(0.5)

        education = check_condition('학력')
        time.sleep(0.5)

        salary = check_condition('급여조건')
        time.sleep(0.5)

        preferred = check_condition('우대사항')
        time.sleep(0.5)
        
        company_field = driver.find_element(By.XPATH, '//*[@id="local_m03"]/div/div/ul[1]/li[3]/div[2]/em').text
            
        try:
            body_url = driver.find_element(By.XPATH, '//*[@id="ifrmJobCont"]').get_attribute('src')
            
        #body에 해당하는 xpath가 존재하지 않으면 내용자체가 없는 공고로 의미 없는 데이터라 생각하여 미수집    
        except:
            continue
            
        driver.get(body_url)
        driver.implicitly_wait(2)

        body_text = text_recruit()
        time.sleep(0.5)
        
        body_img = image_recruit()

        new_row = {'직무명': list(jl.items())[0][0], '채용공고명': head, '해시태그': job_tag,
                   '채용공고상세_text': body_text, '채용공고상세_img' : body_img, '채용공고상세_url' : None,
                   '고용형태' : emptype, '경력' : career, '근무지역': location,
                   '학력' : education, '급여조건' : salary, '우대사항' : preferred,
                   '회사명': corp, '회사분야': company_field, '출처': recruit_url}

        df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)
    
    p += 1
    
#=['직무명', '채용공고명', '해시태그','채용공고상세','고용형태', '경력', '근무지역', '학력', '급여조건', '우대사항',
#                             '회사명', '회사분야', '출처']
# driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[2]/div[2]/div[2]')

---------- 1page ----------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:45<00:00, 16.56s/it]


---------- 2page ----------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:55<00:00, 17.53s/it]


---------- 3page ----------


...

---------- 31page ----------


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:05<00:00, 18.57s/it]


---------- 32page ----------


 30%|████████████████████████▉                                                          | 3/10 [01:01<02:24, 20.60s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[4]/div[1]/em"}
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A5A813+48355]
	(No symbol) [0x009EC4B1]
	(No symbol) [0x008F5358]
	(No symbol) [0x009209A5]
	(No symbol) [0x00920B3B]
	(No symbol) [0x0094E232]
	(No symbol) [0x0093A784]
	(No symbol) [0x0094C922]
	(No symbol) [0x0093A536]
	(No symbol) [0x009182DC]
	(No symbol) [0x009193DD]
	GetHandleVerifier [0x00CBAABD+2539405]
	GetHandleVerifier [0x00CFA78F+2800735]
	GetHandleVerifier [0x00CF456C+2775612]
	GetHandleVerifier [0x00AE51E0+616112]
	(No symbol) [0x009F5F8C]
	(No symbol) [0x009F2328]
	(No symbol) [0x009F240B]
	(No symbol) [0x009E4FF7]
	BaseThreadInitThunk [0x76D77D59+25]
	RtlInitializeExceptionChain [0x77A8B74B+107]
	RtlClearBits [0x77A8B6CF+191]


In [36]:
if driver.find_elements(By.CLASS_NAME, 'error-img'):
    print('공고없음')

공고없음


In [37]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True) #창을 보면서 작업할 수 있음
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
https://job.incruit.com/jobdb_info/jobpost.asp?job=2108310001407

In [38]:
driver.get('https://job.incruit.com/jobdb_info/jobpost.asp?job=2108310001407')

In [39]:
driver.find_elements(By.CLASS_NAME, 'error-img')

[]

In [29]:
driver.find_element(By.XPATH, '//*[@id="ifrmJobCont"]')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ifrmJobCont"]"}
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A5A813+48355]
	(No symbol) [0x009EC4B1]
	(No symbol) [0x008F5358]
	(No symbol) [0x009209A5]
	(No symbol) [0x00920B3B]
	(No symbol) [0x0094E232]
	(No symbol) [0x0093A784]
	(No symbol) [0x0094C922]
	(No symbol) [0x0093A536]
	(No symbol) [0x009182DC]
	(No symbol) [0x009193DD]
	GetHandleVerifier [0x00CBAABD+2539405]
	GetHandleVerifier [0x00CFA78F+2800735]
	GetHandleVerifier [0x00CF456C+2775612]
	GetHandleVerifier [0x00AE51E0+616112]
	(No symbol) [0x009F5F8C]
	(No symbol) [0x009F2328]
	(No symbol) [0x009F240B]
	(No symbol) [0x009E4FF7]
	BaseThreadInitThunk [0x76D77D59+25]
	RtlInitializeExceptionChain [0x77A8B74B+107]
	RtlClearBits [0x77A8B6CF+191]


In [47]:
df.to_csv('C:/datafile/크롤링_incruit_sample.csv', encoding = 'utf-8-sig', index = False)

In [44]:
df

,직무명,채용공고명,해시태그,채용공고상세_text,채용공고상세_img,채용공고상세_url,고용형태,경력,근무지역,학력,급여조건,우대사항,회사명,회사분야,출처
0,웹개발,[SK마이크로웍스/SK마이크로웍스솔루션즈] 2023년도 3분기 신입/경력 구성원 영입,"웹디자인, 생산관리, 연구원·연구개발, 전자·반도체, 웹프로그래머, HTML·퍼블리...","※ SK그룹 공채와 중복 지원 가능 (대졸 초임 4,800 여 만원)",https://f.incru.it/recruiter/jobpost/000496085...,None,정규직,신입/경력(연차무관),"충남 > 전지역, 충북 > 전지역, 진천군, 경기 > 전지역, 수원시 팔달구, 수원...",대졸이상,회사내규,None,SK마이크로웍스솔루션즈 주식회사,전기·전자·반도체,https://job.incruit.com/jobdb_info/jobpost.asp...
1,웹개발,23년 6월 비상교육 직무별 신입/경력 대규모 인재채용,"빅데이터·AI(인공지능), 응용프로그래머, ERP·시스템분석·설계, 마케팅, 경영·...",,https://c.incru.it/newjobpost/2023/06_visang/v...,None,정규직,신입/경력(연차무관),"서울 > 구로구, 전지역, 전국 > 전지역",학력무관,회사내규,None,(주)비상교육,교육·연구및개발,https://job.incruit.com/jobdb_info/jobpost.asp...
2,웹개발,2023년 IBK연금보험 경력직 공개채용 (퇴직연금 연금계리·퇴직연금영업·자산운용 ...,"전산·IT기술지원, 금융사무·심사·추심, 세무·회계, 경영·기획·전략, 일반사무·사...","모집분야 및 모집인원, 담당업무 (세부내용은 아래의 모집분야별 직무기술서 참조) 퇴...",https://c.incru.it/newjobpost/2023/06_ibk/ibk....,None,정규직,경력4년이상,"서울 > 중구, 전지역, 전국 > 전지역, 경기 > 전지역, 인천 > 전지역, 전국...",학력무관,회사내규,None,아이비케이연금보험(주),보험·연금,https://job.incruit.com/jobdb_info/jobpost.asp...
3,웹개발,코인 거래소 개발자 채용 공고 / 경력,"웹프로그래머, ERP·시스템분석·설계, 응용프로그래머",빅파크 코인 거래소 개발자 채용 공고 / 경력 모집부문 및 자격요건 모집부문 담당...,None,None,정규직,신입/경력(연차무관),경기 > 부천시,학력무관,"월급 5,000,000원",None,빅파크,쇼핑몰·전자상거래·오픈마켓,https://job.incruit.com/jobdb_info/jobpost.asp...
4,웹개발,전기 사용 모니터링 앱/웹 개발 경력 채용,"HTML·퍼블리싱·UI개발, 웹프로그래머",아이씨케이 전기 사용 모니터링 앱/웹 개발 경력 채용 모집부문 및 자격요건 모집부...,None,None,"정규직 [수습기간: 1개월], 계약직 [근무기간: 협의 | 정규직 전환검토], 프리...",경력1년이상,서울 > 중랑구,초대졸이상,협의,"전공계열 : 공학계열, 자연과학계열",아이씨케이,전기·전자·반도체,https://job.incruit.com/jobdb_info/jobpost.asp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,웹개발,"[파크앤소프트] WEB(Java, Spring) 개발자를 모십니다.","웹프로그래머, 데이터베이스·DBA","(주)파크앤소프트 [파크앤소프트] WEB(Java, Spring) 개발자를 모십니다...",None,None,정규직 [수습기간: 0개월],경력3년이상,서울 > 송파구,학력무관,협의,[OFFICE 문서] 워드프로세서 류,(주)파크앤소프트,소프트웨어·솔루션·ASP,https://job.incruit.com/jobdb_info/jobpost.asp...
303,웹개발,"[메가존클라우드] Technical Account Manager, AWS","ERP·시스템분석·설계, 전산·IT기술지원, 응용프로그래머, 웹프로그래머",[Cloud Technology]센터를 소개합니다! CTC(Cloud Technol...,None,None,정규직,경력6~10년,서울 > 강남구,학력무관,회사내규,None,메가존(주),웹에이젼시·IT컨설팅,https://job.incruit.com/jobdb_info/jobpost.asp...
304,웹개발,[메가존클라우드] Web Publisher,"빅데이터·AI(인공지능), 웹프로그래머",[Hybrid & AI Platform Center]를 소개합니다! 메가존클라우드의...,None,None,정규직,경력3년이상,서울 > 강남구,학력무관,회사내규,None,메가존(주),웹에이젼시·IT컨설팅,https://job.incruit.com/jobdb_info/jobpost.asp...
305,웹개발,그룹웨어 개발 및 클라우드 컨설팅 기술지원 모집,"웹프로그래머, 응용프로그래머",가온아이 Java 웹개발 신입/경력 모집합니다. 모집부문 및 자격요건 모집분...,https://i.incru.it/jobTemplet/temp2018/img/tem...,None,정규직,신입/경력(연차무관),서울 > 강남구,고졸이상,협의,None,가온아이,소프트웨어·솔루션·ASP,https://job.incruit.com/jobdb_info/jobpost.asp...
